In [1]:
import sys
import os
import numpy as np

from codingTools import *


In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
#originalFile="../images/03508649.1_512_512_2_0_12_0_0_0d.raw" #int16
originalFile="../images/03508649.1_128_128_1_0_8_0_0_0.raw"
#originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
#ImageRaw = load_image_raw(originalFile, 1, 512, 512, "int16") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
ImageRaw = load_image_raw(originalFile, 1, 128, 128, "uint8")
#ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics_data = statistics(original_image)
entropy = statistics_data.compute_entropy()
print("order zero entropy of the image: ",entropy)

order zero entropy of the image:  6.7004541529645625


In [4]:
######SAVE RAW IMAGE######
# decodedFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0_copy.raw"
# ImageRawDecoded = save_image_raw(decodedFile)
# ImageRawDecoded.save_image_raw(original_image,"uint8")
###########

In [5]:
### Quantize ###
quantize = Quantizer(1, 10)
image_data_quantized = ImageRaw.get_image_structure_empty()
quantize.quantize(original_image, image_data_quantized)
quant_statics = statistics(image_data_quantized).compute_entropy()
print(f"Entropia: {quant_statics}")


Check Quantize: [[[1. 2. 2. ... 2. 2. 1.]
  [1. 2. 2. ... 2. 2. 2.]
  [2. 2. 3. ... 3. 2. 2.]
  ...
  [2. 2. 3. ... 3. 3. 2.]
  [1. 2. 2. ... 3. 2. 2.]
  [1. 2. 2. ... 2. 2. 1.]]]
Entropia: 3.4368520266679963


In [6]:
### Dequantize ###
dequantize = Quantizer(1, 10)
image_data_dequantized = ImageRaw.get_image_structure_empty()
dequantize.dequantize(image_data_quantized, image_data_dequantized)

In [7]:
### PAE ###
pae = Metrics(original_image, image_data_dequantized)
print(f"Pae: {pae.calculate_pae()}")
print(pae.PAE_data)


Pae: 9.0
9.0


In [8]:
### MSE ###
mse = Metrics(original_image, image_data_dequantized)
print(f"Mse: {mse.calculate_mse()}")

Mse: 28.00189208984375


In [9]:
### PSNR ###
psnr = Metrics(original_image, image_data_dequantized)
print(f"Psnr: {psnr.calculate_psnr()} dB")

Psnr: 33.69292552980915 dB


In [11]:
### S-transform forward ###
array = [1,2,3,4,8,6,10,8]
transform = Wavelet(array, 1)
array_transform = transform.s_tranform_forward(array)
print(f"Array modificado: {array_transform}")


Array modificado: [ 2  4  7  9 -1 -1  2  2]


In [12]:
### S-transform Inverse ###
array_inv = transform.s_transform_inverse(array_transform)
print(f"Array invertido: {array_inv}")


Array invertido: [ 1  2  3  4  8  6 10  8]


In [22]:
### S-transform Forward image ###
transform_image = Wavelet(original_image, 1)
#print(len(original_image[0][0]))
transform_forward = transform_image.handle_transform_forward()
print(transform_forward)


Longitud forward: 128
[[146 155 161 ...   4   3   3]
 [150 159 167 ...   4   4   3]
 [153 164 173 ...   5   4   4]
 ...
 [153 163 173 ...   3   4   4]
 [149 159 167 ...   3   3   2]
 [146 155 161 ...   3   3   3]]


In [23]:
### S-transform Reverse image ###
transform_image_reverse = Wavelet(transform_forward, 1)
transform_reverse = transform_image_reverse.handle_transform_inverse()
print(transform_reverse)


[[[272  21 281 ...  25  22  19]
  [275  25 285 ...  29  25  22]
  [278  28 290 ...  34  29  25]
  ...
  [278  28 289 ...  34  30  26]
  [275  24 285 ...  30  25  23]
  [272  21 281 ...  25  22  19]]]


In [24]:
print(original_image)
 

[[[16 21 25 ... 25 22 19]
  [19 25 29 ... 29 25 22]
  [22 28 34 ... 34 29 25]
  ...
  [22 28 33 ... 34 30 26]
  [19 24 29 ... 30 25 23]
  [16 21 25 ... 25 22 19]]]


In [27]:
### S-transform forward ###
array = original_image[0][0]
transform = Wavelet(array, 1)
array_transform = transform.s_tranform_forward(array)
print(f"Array modificado: {array_transform}")


Array modificado: [146 155 161  36  37  37  37 166  38  34  28  22  14 144 240 183 178 301
 172 170 165 163 163 158 282 286 154 150 148 276 148 140 265 137 135 133
 121 250 114 249 120 246 120 247 247 246 251 116  46 151 154 159 162 165
 167  39  36  36 165  37  37  33  26  20 251 252 253   1   0   0   1 254
   1   3   3   4   3 244 178   1   5 255   1   2   3   0   0   2 255 255
   2   1   4 255   2   6 252   1   3   3   4 253   3 254   0 255   1 255
 255 255 254  17  32 254 255 254 254 255 255   1   1   0 255   0   0   4
   3   3]


In [28]:
### S-transform Inverse ###
array_inv = transform.s_transform_inverse(array_transform)
print(f"Array invertido: {array_inv}")


Array invertido: [272  21 281  29 288  35  37  36  37  37  37  37  38  37 293  39  39  38
  36  33  30  27  24  20  16  13 266  22 329 151 184 183 181 176 429 174
 173 172 171 169 167 164 163 163 163 163 159 157 410 155 414 159 155 153
 151 150 150 146 404 149 149 147 143 137 391 139 138 137 137 134 135 132
 123 119 377 124 116 113 376 122 120 120 374 119 121 120 375 120 375 120
 374 119 378 124 125 108  62  30 278  24 282  27 286  32 289  35 293  38
 295  40  40  39  37  36  36  36 293  38  37  37  37  37  35  31  28  25
  22  19]


In [29]:
print(f"Array original: {original_image[0][0]}")

Array original: [ 16  21  25  29  32  35  37  36  37  37  37  37  38  37  37  39  39  38
  36  33  30  27  24  20  16  13  10  22  73 151 184 183 181 176 173 174
 173 172 171 169 167 164 163 163 163 163 159 157 154 155 158 159 155 153
 151 150 150 146 148 149 149 147 143 137 135 139 138 137 137 134 135 132
 123 119 121 124 116 113 120 122 120 120 118 119 121 120 119 120 119 120
 118 119 122 124 125 108  62  30  22  24  26  27  30  32  33  35  37  38
  39  40  40  39  37  36  36  36  37  38  37  37  37  37  35  31  28  25
  22  19]


### Classe 17/09
Predicció:
X = 1,2,3,4,8,6,10,8
Causal
Pi = {0 si i=0, X(i-1) si i>0}
X'i = Xi - Pi
P = 0,1,2,3,4,8,6,10 (Predicció)
X' = 1,1,1,1,4,-2,4,-2
#
###### TODO
Predict sobre una matriu. -> ha de ser invertible -> img de 512 512
Calcular entropia del resultat X' -> H(x') <= H(x)

In [17]:
var = -100.9
print(int(var))

-100


In [18]:
import numpy as np

# Crear una matriz de 8x8 con los números del 1 al 8 en cada fila
matriz_2d = np.tile(np.arange(1, 17), (16, 1))
print(matriz_2d)
# Crear una matriz de 3 dimensiones repitiendo la matriz 2D
matriz_3d = np.expand_dims(matriz_2d, axis=0)
print(matriz_3d)


[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]]
[[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8 

In [19]:
### S-transform Forward image ###
transform_image = Wavelet(matriz_3d, 3)
#print(len(original_image[0][0]))
transform_forward = transform_image.handle_transform_forward()
print(transform_forward)

Longitud forward: 16
Longitud forward: 8
Longitud forward: 4
[[ 5 13 -4 -4 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 5 13 -4 -4 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 5 13 -4 -4 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 5 13 -4 -4 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 3  7 11 15 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 3  7 11 15 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 3  7 11 15 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 3  7 11 15 -2 -2 -2 -2 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]
 [ 2  4  6  8 10 12 14 16 -1 -1 -1 -1 -1 -1 -1 -1]]


In [20]:
### S-transform Reverse image ###
transform_image_reverse = Wavelet(transform_forward, 3)
transform_reverse = transform_image_reverse.handle_transform_inverse()
print(transform_reverse)

[[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]]]


In [21]:
print(matriz_3d)

[[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]]]
